In [ ]:
#####################################   Importing data   #########################################################################################

#Importing the numpy library
import numpy as np

#Importing functions from the vpython library
from vpython import *

%matplotlib notebook
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display
from matplotlib.widgets import Slider, Button

###############################################################################################################################

In [ ]:
########################################   GAME   ############################################################################################################################################

# mixing factors (converted from degrees to radians)
th12 = np.radians(33.82)
th23 = np.radians(48.3)
th13 = np.radians(8.61)
delta_CP = np.radians(222)

# mass differences
dm21_sq = 7.53*10**(-5) # eV^2
dm32_sq = 2.52*10**(-3) # eV^2

# mass eigenstates
m1 = 0.75 # eV
m2 = np.sqrt(m1*m1 + dm21_sq) # eV
m3 = np.sqrt(m2*m2 + dm32_sq) # eV

#calculate third mass difference squared
dm31_sq = m3*m3 - m1*m1 # eV^2

# set energy values
E_sun_pp = 4*10**5 # eV 
E_atm=10**11 #eV
E_acc=10**9 #eV

# important constants:
# speed of light
c = 299792.458 # km s^-1 
# Planck constant
hbar = 6.582119569*10**(-16) # eV*s
# Astronomical unit (i.e. the distance between the Sun and the Earth)
AU = 149597870.700 # km



def prob_transition(alpha, beta, E, L):
    """Calculates the probability of a transition between an initial neutrino alpha to a neutrino beta
    Takes as inputs the initial and final neutrino flavours alpha and beta, 
    the energy at which the neutrino is produced E (in eV),
    and the distance travelled by the particle L (in km)
    Returns the probability of the transition"""
    
    if (alpha == "electron" and beta == "muon") or (alpha == "muon" and beta == "electron"): 
        ar = np.array([th12, dm21_sq])
        arg = 1.27*ar[1]*L/E
        P = np.sin(2*ar[0])*np.sin(2*ar[0])*np.sin(arg)*np.sin(arg)
        
    elif (alpha == "electron" and beta == "tau") or (alpha == "tau" and beta == "electron"):
        ar = np.array([th13, dm31_sq])
        arg = 1.27*ar[1]*L/E
        P = np.sin(2*ar[0])*np.sin(2*ar[0])*np.sin(arg)*np.sin(arg)
        
    elif (alpha == "muon" and beta == "tau") or (alpha == "tau" and beta == "muon"):
        ar = np.array([th23, dm32_sq])
        arg = 1.27*ar[1]*L/E
        P = np.sin(2*ar[0])*np.sin(2*ar[0])*np.sin(arg)*np.sin(arg)
        
    elif (alpha == beta):
        
        if (alpha == "electron"):
            ar1 = np.array([th12, dm21_sq])
            ar2 = np.array([th13, dm31_sq])
            
        elif (alpha == "muon"):
            ar1 = np.array([th12, dm21_sq])
            ar2 = np.array([th23, dm32_sq])
            
        elif (alpha == "tau"):
            ar1 = np.array([th13, dm31_sq])
            ar2 = np.array([th23, dm32_sq])
        
        arg1 = 1.27*ar1[1]*L/E
        arg2 = 1.27*ar2[1]*L/E
        
        P1 = np.sin(2*ar1[0])*np.sin(2*ar1[0])*np.sin(arg1)*np.sin(arg1)
        P2 = np.sin(2*ar2[0])*np.sin(2*ar2[0])*np.sin(arg2)*np.sin(arg2)
        
        if ((P1 + P2) < 1) :
            P = 1 - (P1 + P2)
            
        else:
            P = 0
    
    else:
        return "Error"
    
    return P



####### Setting up initial data #######

g = 30 
mass_bird = 1
radius_bird = 0.5
t_applied = 0
p = 200
contact_time = 0.01 
dt = 0.005
d_angle = 0.01
x0 = 0.0
y0 = 0.0
run = 0

bird_color = [color.blue,color.green,color.red]
bird_mass_tople = [0.3,1,1.8]



####### Defining the functions #######

def trajectory(x_pos):
    '''
    Calculates the trajectory (y coordinate) at a given horizontal distance (x coordinate) from the lauch point.
    
    Input: 
     - x_pos: x coordinate of the point where the trajectory is required.
     
    Output:
     - trajectory: y coordinate of the particle at this point.
    '''
    
    trajectory = -g*(x_pos-x0)**2 / (2*v0**2*np.cos(theta)**2) + np.tan(theta)*(x_pos-x0) + y0
    
    return trajectory    


####### Main code of the game ####### 

V_center = vector(8,0,0)
V_camera = vector(4,2,12)
V_title = vector(2,6,-2)
  
# Setting up the scene 
scene = canvas(width=1000, height=700, center=V_center, range=8, background = color.cyan)
scene.camera.pos = V_camera
scene.forward = V_center - scene.camera.pos
scene.camera.axis = V_center - scene.camera.pos
scene.userzoom = False
scene.userspin = False
    
# Ground 
ground = box(pos= V_center, length=100, height=0.1, width=100, texture = textures.metal)
    
# Target
target = box(pos=vector((2*random()+1)*7,1,0), axis=vector(1,0,0), length= 0.5, height= 3 + random(), width=0.5, color = color.black)

# Initial Bird
bird = sphere(pos = vector(x0,y0,0), radius = 0.2, color=color.blue, make_trail = True, trail_color = color.blue)

# Text
label_title = label(text = 'ANGRY NEUTRINOS',pos = vector(8,7,0), color = color.black, box = False, background = color.cyan , height = 40, opacity = 0)
label_level = label(text = 'Level 1' , pos = vector(8,6,0), color = color.black, box = False, background = color.cyan , height = 30, opacity = 0)
        

def setV(v):
    global v0
    wt1.text = '{:1.0f}'.format(v.value)
    v0 = v.value
    
sl1 = slider(min = 0 , max = 50 , value = 0, length = 220 , bind = setV, right = 15)
wt1 = wtext(text = '{:1.0f}'.format(sl1.value))
    
    
    
    
def setT(t):
    global theta
    wt2.text = '{:1.0f}'.format(t.value)
    theta = np.radians(t.value)
    
sl2 = slider(min = 0 , max = 90 , value = 0, length = 220 , bind = setT, right = 15)
wt2 = wtext(text = '{:1.0f}'.format(sl2.value)) 



def launch():
    run = 1
    
launch_button = button(text= 'Launch' , color = color.red , background = color.white, pos = scene.title_anchor, bind = launch)


# Initial data with respect to the scene set up 
mass_target = p * (target.length*target.width*target.height)
hit_tolerance = bird.radius + target.length/2
x_impact = target.pos.x - hit_tolerance
t_restoring = mass_target*g*target.length/2 


while t_restoring > t_applied:

    bird.clear_trail()
    del bird
    
    bird = sphere(pos = vector(x0,y0,0), radius = 0.2, color=color.blue, make_trail = True, trail_color = color.blue)
    
    
    while (run != 1):
        run = run
    
    
    if target.height + bird.radius > trajectory(x_impact) > 0 :
        
        x = x0
        y = y0
        t = 0
        distance_covered = 0

        while x <= x_impact :
            rate(60)
            t += dt

            distance_covered += np.sqrt(abs(x - (x0 + v0*t*np.cos(theta)))**2 + abs(y - (y0 + v0*t*np.sin(theta) - 0.5*g*t**2))**2)

            prob_elec = prob_transition("muon", "electron", E_sun_pp , distance_covered*AU)
            prob_muon = prob_transition("muon", "muon", E_sun_pp , distance_covered*AU)
            prob_tau = prob_transition("muon", "tau", E_sun_pp , distance_covered*AU)

            prob = [prob_muon,prob_tau,prob_elec]
            prob_max = np.amax(prob)
            index_max = prob.index(prob_max)

            mass_tople = bird_mass_tople[index_max]
            bird.color = bird_color[index_max]
            bird.trail_color = bird_color[index_max]

            x = x0 + v0*t*np.cos(theta) 
            y = y0 + v0*t*np.sin(theta) - 0.5*g*t**2 
            bird.pos = vector(x,y,0)
            

        c_rot = target.pos + vector(target.length/2,-target.height/2,0)
        bird_momentum = vector(mass_bird*v0*np.cos(theta) , mass_bird*v0*np.sin(theta) - mass_bird*g*t , 0)
        vector_da = vector(x_impact,trajectory(x_impact),0) - c_rot
        t_applied_vector = cross(bird_momentum/contact_time,vector_da)
        t_applied = mag(t_applied_vector)

        if t_applied > t_restoring :

            s = target.pos
            w = bird.pos
            alpha = 0
            betha = np.pi - np.arccos((target.length/2)/mag(s-c_rot))
            gamma = np.pi - np.arccos((hit_tolerance + target.length/2) / mag(w-c_rot))

            x = x_impact
            y = trajectory(x_impact)
            t = 0

            while alpha >= -np.pi/2:
                rate(100)
                betha -= d_angle
                alpha -= d_angle
                gamma -= d_angle
                o = vector(np.cos(alpha),np.sin(alpha),0)
                u = c_rot + vector(np.cos(betha),np.sin(betha),0)*mag(s-c_rot)
                z = c_rot + vector(np.cos(gamma),np.sin(gamma),0)*mag(w-c_rot)
                target.pos = u + vector(0,target.width,0)
                target.axis = o
                bird.pos = z

        else :
    
            x = x_impact
            y = trajectory(x_impact)
            t = 0

            while y > 0 :
                rate(60)
                t += dt
                y = -g*t**2 + trajectory(x_impact)
                bird.pos = vector(x,y,0)


    else :
        
        x = x0
        y = y0
        t = 0
        distance_covered = 0

        while x < 25 and y >= 0:
            rate(60)
            t += dt

            distance_covered += np.sqrt(abs(x - (x0 + v0*t*np.cos(theta)))**2 + abs(y - (y0 + v0*t*np.sin(theta) - 0.5*g*t**2))**2)

            prob_elec = prob_transition("muon", "electron", E_sun_pp , distance_covered*AU)
            prob_muon = prob_transition("muon", "muon", E_sun_pp , distance_covered*AU)
            prob_tau = prob_transition("muon", "tau", E_sun_pp , distance_covered*AU)

            prob = [prob_muon,prob_tau,prob_elec]
            prob_max = np.amax(prob)
            index_max = prob.index(prob_max)

            mass_tople = bird_mass_tople[index_max]
            bird.color = bird_color[index_max]
            bird.trail_color = bird_color[index_max]

            x = x0 + v0*t*np.cos(theta)
            y = y0 + v0*t*np.sin(theta) - 0.5*g*t**2 
            bird.pos = vector(x,y,0)

##################################################################################################################################################################################